In [65]:
import pandas as pd
import numpy as np
import math
import os

# import for time, and file change
import json
from PIL import Image

# frequent updating
import time
import random

In [66]:
# define filepaths
workspace = "/Users/advaysingh/Documents/projects/hindi_classification/" 
#print("Current workspace:", workspace)

data = os.path.join(workspace, 'data/Hindi/')
dict_lib = os.path.join(workspace, 'data/dict.csv')
img_path = os.path.join(workspace, 'server/snapshot.png')
out_file = os.path.join(workspace, 'server/outputs.json')


In [67]:
# create train/test dicts with files and labels

def create_dict(x: str) -> dict:
    x_dict = {}
    i = 0
    for dir in os.listdir(os.path.join(data, x)):
        for file in os.listdir(os.path.join(data, x, dir)):
            x_dict[os.path.join(data, x, dir, file)] = i
        i += 1
    return x_dict

# make pandas df

train_df = pd.DataFrame.from_dict(create_dict('Train'), orient='index')
test_df = pd.DataFrame.from_dict(create_dict('Test'), orient='index')

train_df.head()

,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/26446.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/62562.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/74300.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/86182.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/62576.png,0


In [68]:
# Activations class
class Activation:
    # can add more types
    def __init__(self, act_type: str) -> None:
        self.act = act_type

    def print_act(self) -> None:
        return self.act
    
    def compute(self, z: float) -> float:

        # Sigmoid function
        if (self.act == 'sigmoid'):
            return 1 / (1 + math.exp(z))
        
        # ReLU function
        elif (self.act == 'relu'):
            if (z < 0): return 0
            return z

    def prime(self, z: float) -> float:

        # Sigmoid prime
        if (self.act == 'sigmoid'):
            return (self.compute(z))(1 - self.compute(z))
        
        # ReLU prime
        if (self.act == 'relu'):
            if (z < 0): return 0
            return 1


def softmax(i: int, costs: list) -> float:
    return costs[i] / sum(costs)

def softmax_prime(i: int, costs: list) -> float:
    softmax(i, costs)(1 - softmax(i, costs))


    
a = Activation('sigmoid')

print(a.compute(233.2))

5.278698047466735e-102


In [69]:
class Model:
    def __init__(self, train_df: pd.DataFrame, test_df: pd.DataFrame,
                  act: Activation, layers: int, step: float, stochastic: bool) -> None:
        self.train_df = train_df
        self.test_df = test_df

        # create activation function
        self.activation = act

        # init weight and bias with # layers np arrays
        self.weight_arrs = [None] * layers
        self.bias_arrs = [None] * layers

        # init learn speed
        self.speed = step

        # determine gradient decent type
        self.stochastic = stochastic

    def img_to_np(self, dir) -> np.array:
        return np.array(Image.open(dir)).flatten()
    
    def print_img(self) -> None:
        for img_dir, row in self.train_df.iterrows():
            print(self.img_to_np(img_dir), row[0])
    




In [70]:
cnn = Model(train_df.head(), test_df.head(), Activation('sigmoid'), 3, 1.2, False)
cnn.print_img()

[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0


In [71]:
import json
import time
import random

from PIL import Image


def update_json_file(hindi_value, english_value):
    with open(out_file, 'r') as file:
        data = json.load(file)
    data['hindi'] = hindi_value
    data['english'] = english_value
    with open(out_file, 'w') as file:
        json.dump(data, file, indent=4)

# Define a list of possible values if applicable
hindi_values = ["हिं", "नया", "मूल्य", "उदाहरण"]
english_values = ["Eng", "New", "Value", "Example"]

# Loop to update the JSON file every second with a random value
try:
    while True:
        # Generate a random value from the lists
        new_hindi_value = random.choice(hindi_values)
        new_english_value = random.choice(english_values)

        update_json_file(new_hindi_value, new_english_value)
        
        # Print the new values (for debug purposes)
        # print(f'Updated JSON file with hindi: {new_hindi_value}, english: {new_english_value}')

        # read in and resize img
        img = Image.open(img_path).convert("L").resize((32, 32))
 
        # read in image for debugging
        """img.show("img to classify")
        print(img.format)
        print(img.mode)
        print(img.size)"""

        # create np array and resize
        img_data = np.array(img).flatten()
        print(img_data.shape)
        print(img_data)

        
        time.sleep(10)  # Wait for 1 second
except KeyboardInterrupt:
    print("Update stopped by user.")

(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[67 11 10 ...  0  0  0]
Update stopped by user.
